In [5]:
import torch
import torch.nn as nn
from torch_geometric.data import Data, DataLoader
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATConv, global_mean_pool, GCNConv
import glob
import numpy as np
import matplotlib.pyplot as plt
import os
import torch.nn.functional as F
import pandas as pd
import json

# Define GAT model for batched data
class GAT(torch.nn.Module):
    def __init__(self, in_channels, out_channels, dropout_p=0.1):
        super().__init__()
        self.gat = GATConv(in_channels, out_channels, heads=1, concat=True, edge_dim=1)
        self.pool = global_mean_pool  # Can also use global_max_pool or global_add_pool
        self.dropout = nn.Dropout(p=dropout_p)
        self.norm = nn.BatchNorm1d(out_channels)
        self.linear = torch.nn.Linear(out_channels, 1)

    def forward(self, x, edge_index, edge_attr, batch):
        out, attn_weights = self.gat(x, edge_index, edge_attr, return_attention_weights=True)
        out = self.dropout(out)
        out = self.pool(out, batch)  # Pool over nodes in each graph
        out = self.norm(out)
        out = self.dropout(out) 
        out = self.linear(out)
        return out, attn_weights

def gat_organize_graph_and_add_weight(file_path, label):
    data = np.load(file_path, allow_pickle=True).item()
    inverse_distance = data['inverse_distance']
    encoded_matrix = data['encoded_matrix']

    x = torch.tensor(encoded_matrix, dtype=torch.float32)
    adj = torch.tensor(inverse_distance, dtype=torch.float32)

    # Normalize adjacency (row-normalize)
    adj = adj / (adj.sum(dim=1, keepdim=True) + 1e-8)

    # Create edge_index and edge weights
    edge_index = (adj > 0).nonzero(as_tuple=False).t()
    edge_weight = adj[adj > 0]

    y = torch.tensor([label], dtype=torch.float32)
    
    return Data(x=x, edge_index=edge_index, edge_attr=edge_weight, y=y)

class GCN(nn.Module):
    def __init__(self, input_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, 32)
        self.bn1 = nn.BatchNorm1d(32)
        
        self.conv2 = GCNConv(32, 64)
        self.bn2 = nn.BatchNorm1d(64)
        
        self.conv3 = GCNConv(64, 128)
        self.bn3 = nn.BatchNorm1d(128)

        self.dropout_gcn = nn.Dropout(0.2)
        self.dropout = nn.Dropout(0.6)
        
        self.fc1 = nn.Linear(128, 64)
        self.out = nn.Linear(64, 1)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index, data.edge_attr, data.batch

        x = self.conv1(x, edge_index, edge_weight)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout_gcn(x)

        x = self.conv2(x, edge_index, edge_weight)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout_gcn(x)

        x = self.conv3(x, edge_index, edge_weight)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.dropout_gcn(x)

        # Global pooling to get graph-level representation
        x = global_mean_pool(x, batch)

        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.out(x)

        return x
    
def gcn_organize_graph_and_add_weight(file_path, label):
    data = np.load(file_path, allow_pickle=True).item()
    inverse_distance = data['inverse_distance']
    encoded_matrix = data['encoded_matrix']

    x = torch.tensor(encoded_matrix, dtype=torch.float32)
    adj = torch.tensor(inverse_distance, dtype=torch.float32)

    # Normalize adjacency (row-normalize)
    #adj = adj / (adj.sum(dim=1, keepdim=True) + 1e-8)

    # Create edge_index and edge weights
    edge_index = (adj > 0).nonzero(as_tuple=False).t()
    edge_weight = adj[adj > 0]

    y = torch.tensor([label], dtype=torch.float32)
    
    return Data(x=x, edge_index=edge_index, edge_attr=edge_weight, y=y)

# Define the 2D CNN model in PyTorch
class CNN2D(nn.Module):
    def __init__(self, input_channels):
        super(CNN2D, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(input_channels, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU()
        )
        self.pool3 = nn.MaxPool2d(2, 2)
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 25 * 8, 128)

        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = self.pool3(x)
        
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x
    
class CNN3D(nn.Module):
    def __init__(self):
        super(CNN3D, self).__init__()
        
        self.conv0 = nn.Conv3d(in_channels=65, out_channels=64, kernel_size=1, stride=1, padding=0) # play around with output channels
        self.conv1 = nn.Conv3d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv3d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool3d(kernel_size=2, stride=2, padding=0)

        #self.dropout_conv = nn.Dropout3d(p=0.05)
        
        # After two pooling layers, spatial dimensions reduce from 40x40x40 -> 5x5x5
        self.fc1 = nn.Linear(128 * 4 * 4 * 4, 256)  # Try increasing over 256
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 2)  # Assuming 1 output for docking status/position

        #self.dropout_fc = nn.Dropout(p=0.15)
        
    def forward(self, x):
        # Forward pass through Conv layers
        x = self.pool(torch.relu(self.conv0(x)))  # Conv0 -> ReLU -> Pooling
        #x = self.dropout_conv(x)
        x = self.pool(torch.relu(self.conv1(x)))  # Conv1 -> ReLU -> Pooling
        x = self.pool2(torch.relu(self.conv2(x)))  # Conv2 -> ReLU -> Pooling

        # Flatten the input for fully connected layers
        x = x.view(-1, 128 * 4 * 4 * 4)
        
        # Forward pass through fully connected layers
        x = torch.relu(self.fc1(x)) #use tanh activation
        #x = self.dropout_fc(x)
        x = torch.relu(self.fc2(x))
        x = torch.nn.functional.softmax(self.fc3(x), dim=1)  # Final layer (output layer)
        #x = torch.clamp(x, min=1e-7, max=1 - 1e-7)  # Clamp outputs to avoid extreme values
        
        return x

In [6]:
# -----------------------------
# Per-experiment spies dirs
# -----------------------------
gat_gcn_spies_exps = [
    "../CLR_Ligand_Data/cholesterol-separate-graphs-clr_exp1/Spies",
    "../CLR_Ligand_Data/cholesterol-separate-graphs-clr_exp2/Spies",
    "../CLR_Ligand_Data/cholesterol-separate-graphs-clr_exp3/Spies",
    "../CLR_Ligand_Data/cholesterol-separate-graphs-clr_exp4/Spies",
    "../CLR_Ligand_Data/cholesterol-separate-graphs-clr_exp5/Spies",
]

gnn_spies_exps = [
    "../CLR_Ligand_Data/cholesterol-graph-clr_exp1/Spies",
    "../CLR_Ligand_Data/cholesterol-graph-clr_exp2/Spies",
    "../CLR_Ligand_Data/cholesterol-graph-clr_exp3/Spies",
    "../CLR_Ligand_Data/cholesterol-graph-clr_exp4/Spies",
    "../CLR_Ligand_Data/cholesterol-graph-clr_exp5/Spies",
]

cnn_spies_exps = [
    "../CLR_Ligand_Data/total-grid-5A-clr_exp1/Spies",
    "../CLR_Ligand_Data/total-grid-5A-clr_exp2/Spies"
]

# eval dirs (these typically don't change per exp)
gat_gcn_unlabeled = "Data/cholesterol-separate-sep-clr/unlabeled"
gnn_unlabeled     = "Data/cholesterol-sep-clr/unlabeled"
cnn_unlabeled     = "Data/total-sep-5A-clr/unlabeled"

OUT_ROOT = "./PU_EvalOutputs_Ensemble"
os.makedirs(OUT_ROOT, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [7]:
# -----------------------------
# File helpers
# -----------------------------
def list_npy_files(dir_path: str):
    return sorted(glob.glob(os.path.join(dir_path, "*.npy")))

def load_npy_dict(file_path: str):
    return np.load(file_path, allow_pickle=True).item()

def load_cnn_matrix(file_path: str) -> np.ndarray:
    x = np.load(file_path, allow_pickle=True)
    # Must be a numeric 2D matrix
    if not isinstance(x, np.ndarray):
        raise ValueError(f"CNN file did not load as ndarray: {file_path}, type={type(x)}")
    if x.ndim != 2:
        raise ValueError(f"CNN matrix must be 2D: {file_path}, got shape {x.shape}")
    if x.dtype == object:
        raise ValueError(f"CNN matrix should not be dtype=object: {file_path}, got dtype=object with shape {x.shape}")
    return x

def infer_graph_input_dim(dir_path: str) -> int:
    files = list_npy_files(dir_path)
    if not files:
        raise FileNotFoundError(f"No .npy files found in: {dir_path}")
    d = load_npy_dict(files[0])
    return int(d["encoded_matrix"].shape[1])

def organize_graph(file_path: str, normalize_adj: bool) -> Data:
    data = np.load(file_path, allow_pickle=True).item()
    adj = torch.tensor(data['inverse_distance'], dtype=torch.float32)
    x = torch.tensor(data['encoded_matrix'], dtype=torch.float32)

    if normalize_adj:
        adj = adj / (adj.sum(dim=1, keepdim=True) + 1e-8)

    edge_index = (adj > 0).nonzero(as_tuple=False).t()
    edge_weight = adj[adj > 0]
    return Data(x=x, edge_index=edge_index, edge_attr=edge_weight)

def ensure_dir(p: str):
    os.makedirs(p, exist_ok=True)

# -----------------------------
# Labeling method (from spies percentiles)
# -----------------------------
def compute_spy_percentiles(spy_probs: np.ndarray):
    p25, p50, p75 = np.percentile(spy_probs, [25, 50, 75])
    return {
        "p25": float(p25),
        "p50": float(p50),
        "p75": float(p75),
        "n_spies": int(len(spy_probs)),
        "min": float(np.min(spy_probs)) if len(spy_probs) else None,
        "max": float(np.max(spy_probs)) if len(spy_probs) else None,
        "mean": float(np.mean(spy_probs)) if len(spy_probs) else None,
        "std": float(np.std(spy_probs)) if len(spy_probs) else None,
    }

def label_from_percentiles(prob: float, stats: dict) -> str:
    p25, p50, p75 = stats["p25"], stats["p50"], stats["p75"]
    if prob <= p25:
        return "Negative"
    elif prob <= p50:
        return "PseudoNegative"
    elif prob <= p75:
        return "PseudoPositive"
    else:
        return "Positive"

def plot_label_histogram(df: pd.DataFrame, title: str, out_png: str):
    order = ["Negative", "PseudoNegative", "PseudoPositive", "Positive"]
    counts = df["label"].value_counts().reindex(order, fill_value=0)
    plt.figure()
    plt.bar(counts.index.tolist(), counts.values.tolist())
    plt.title(title)
    plt.xlabel("Label")
    plt.ylabel("Count")
    plt.xticks(rotation=20, ha="right")
    plt.tight_layout()
    plt.savefig(out_png, dpi=200)
    plt.close()

def resolve_ensemble_checkpoints(pattern_or_dir: str) -> list[str]:
    """
    Give either:
      - a glob pattern:  "/path/to/gat_submodels/*.pt"
      - or a directory: "/path/to/gat_submodels"
    """
    if os.path.isdir(pattern_or_dir):
        ckpts = sorted(glob.glob(os.path.join(pattern_or_dir, "*.pt"))) + \
                sorted(glob.glob(os.path.join(pattern_or_dir, "*.pth")))
    else:
        ckpts = sorted(glob.glob(pattern_or_dir))
    if len(ckpts) == 0:
        raise FileNotFoundError(f"No checkpoints found for: {pattern_or_dir}")
    return ckpts

def load_state_dict_into(model: nn.Module, ckpt_path: str):
    sd = torch.load(ckpt_path, map_location="cpu")
    # supports either raw state_dict or a dict with 'model_state_dict'
    if isinstance(sd, dict) and "model_state_dict" in sd:
        sd = sd["model_state_dict"]
    model.load_state_dict(sd)
    return model

from collections import Counter

def majority_vote(labels: list[str]) -> str:
    c = Counter(labels)
    # deterministic tie-breaker by fixed priority order
    order = ["Negative", "PseudoNegative", "PseudoPositive", "Positive"]
    best = max(order, key=lambda k: (c.get(k, 0), -order.index(k)))
    return best

def run_pu_eval_5exp_with_exp_spies(
    model_name: str,
    out_dir: str,
    spies_dirs: list[str],          # len=2
    eval_dir: str,
    predict_rows_fn_per_exp,        # fn(files, exp_idx)-> rows using THAT exp ckpts
):
    ensure_dir(out_dir)

    eval_files = list_npy_files(eval_dir)
    if len(eval_files) == 0:
        raise FileNotFoundError(f"[{model_name}] no eval files in {eval_dir}")

    exp_eval_probs = []   # list of np.array [n_files] per exp
    exp_eval_labels = []  # list of list[str] per exp
    exp_spy_stats = []

    for exp_idx, spies_dir in enumerate(spies_dirs):
        spy_files = list_npy_files(spies_dir)
        if len(spy_files) == 0:
            raise FileNotFoundError(f"[{model_name}] no spies in {spies_dir}")

        print(f"\n[{model_name}] EXP {exp_idx+1}/2: spies={len(spy_files)} eval={len(eval_files)}")

        # 1) predict spies (with this experiment's ckpts)
        spy_rows = predict_rows_fn_per_exp(spy_files, exp_idx)
        spy_probs = np.array([r["prob"] for r in spy_rows], dtype=float)

        stats = compute_spy_percentiles(spy_probs)
        stats["model"] = model_name
        stats["exp_idx"] = exp_idx + 1
        stats["spies_dir"] = spies_dir
        exp_spy_stats.append(stats)

        # 2) predict eval (with this experiment's ckpts)
        eval_rows = predict_rows_fn_per_exp(eval_files, exp_idx)
        probs = np.array([r["prob"] for r in eval_rows], dtype=np.float64)

        # 3) label eval with THIS experiment's thresholds
        labels = [label_from_percentiles(float(p), stats) for p in probs]

        exp_eval_probs.append(probs)
        exp_eval_labels.append(labels)

        print(f"  thresholds: p25={stats['p25']:.4f}, p50={stats['p50']:.4f}, p75={stats['p75']:.4f}")

    # Stack to [2, n_files]
    exp_eval_probs = np.stack(exp_eval_probs, axis=0)
    prob_mean = exp_eval_probs.mean(axis=0)
    prob_std  = exp_eval_probs.std(axis=0, ddof=0)

    # Majority label vote across the 2 experiments
    labels_majority = []
    for i in range(len(eval_files)):
        labels_i = [exp_eval_labels[e][i] for e in range(2)]
        labels_majority.append(majority_vote(labels_i))

    # Save combined outputs
    out_rows = []
    for i, fp in enumerate(eval_files):
        row = {
            "file": os.path.basename(fp),
            "path": fp,
            "prob": float(prob_mean[i]),
            "prob_std": float(prob_std[i]),
            "label": labels_majority[i],
        }
        # optional: keep per-exp probs for debugging
        for e in range(2):
            row[f"prob_exp{e+1}"] = float(exp_eval_probs[e, i])
        out_rows.append(row)

    df_eval = pd.DataFrame(out_rows)
    out_csv = os.path.join(out_dir, f"{model_name}_eval_probs_and_labels.csv")
    df_eval.to_csv(out_csv, index=False)

    # Save spies stats (per exp)
    pd.DataFrame(exp_spy_stats).to_csv(os.path.join(out_dir, f"{model_name}_spies_stats_2exp.csv"), index=False)
    with open(os.path.join(out_dir, f"{model_name}_spies_stats_5exp.json"), "w") as f:
        json.dump(exp_spy_stats, f, indent=2)

    out_png = os.path.join(out_dir, f"{model_name}_eval_label_hist_5exp.png")
    plot_label_histogram(df_eval, f"{model_name} label counts (mean prob across 2 exps; majority label)", out_png)

    print(f"\n[{model_name}] majority label counts:\n{df_eval['label'].value_counts()}")
    print(f"Saved: {out_csv}")
    print(f"Saved: {out_png}")
    print(f"Saved: {os.path.join(out_dir, f'{model_name}_spies_stats_5exp.json')}")

def load_grid_3d(file_path: str) -> torch.Tensor:
    """
    Loads a (D,H,W,C) numpy grid and returns torch tensor [C,D,H,W] float32.
    Matches your training: torch.tensor(grid).permute(3,0,1,2)
    """
    arr = np.load(file_path)
    if not isinstance(arr, np.ndarray):
        raise ValueError(f"3DCNN grid did not load as ndarray: {file_path}")
    if arr.ndim != 4:
        raise ValueError(f"3DCNN grid must be 4D (D,H,W,C). Got shape {arr.shape} for {file_path}")
    x = torch.tensor(arr, dtype=torch.float32).permute(3, 0, 1, 2)  # [C,D,H,W]
    return x

@torch.no_grad()
def ensemble_predict_probs_cnn3d(
    ckpt_paths: list[str],
    files: list[str],
    batch_size: int = 8,
) -> list[dict]:
    """
    Averages PROBABILITIES across 3DCNN submodels.
    Uses P(class=1) from softmax output.
    """
    if len(files) == 0:
        return []

    prob_sum = np.zeros(len(files), dtype=np.float64)

    for k, ckpt in enumerate(ckpt_paths):
        model = CNN3D().to(device)
        model = load_state_dict_into(model, ckpt)
        model.eval()

        for i in range(0, len(files), batch_size):
            batch_files = files[i:i+batch_size]
            xs = [load_grid_3d(fp) for fp in batch_files]
            x_batch = torch.stack(xs, dim=0).to(device)  # [B,C,D,H,W]

            out = model(x_batch)              # [B,2] softmax
            probs = out[:, 1].detach().cpu().numpy()  # P(class=1)

            prob_sum[i:i+len(probs)] += probs

        print(f"  submodel {k+1}/{len(ckpt_paths)} done: {os.path.basename(ckpt)}")

    prob_avg = prob_sum / float(len(ckpt_paths))
    return [{"file": os.path.basename(fp), "path": fp, "prob": float(prob_avg[i])}
            for i, fp in enumerate(files)]


@torch.no_grad()
def ensemble_predict_probs_graph(
    model_ctor,
    gat_bool,
    ckpt_paths: list[str],
    files: list[str],
    batch_size: int = 16,
    num_workers: int = 0,
) -> list[dict]:

    if len(files) == 0:
        return []

    data_list = [organize_graph(fp, gat_bool) for fp in files]
    loader = DataLoader(data_list, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    prob_sum = np.zeros(len(files), dtype=np.float64)

    for k, ckpt in enumerate(ckpt_paths):
        model = model_ctor().to(device)
        model = load_state_dict_into(model, ckpt)
        model.eval()

        idx = 0
        for batch in loader:
            batch = batch.to(device)

            if isinstance(model, GAT):
                out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
            else:
                out = model(batch)

            logits = out[0] if isinstance(out, (tuple, list)) else out
            logits = logits.view(-1)

            probs = torch.sigmoid(logits).detach().cpu().numpy()

            bsz = len(probs)
            prob_sum[idx:idx+bsz] += probs
            idx += bsz

        print(f"  submodel {k+1}/{len(ckpt_paths)} done: {os.path.basename(ckpt)}")

    prob_avg = prob_sum / float(len(ckpt_paths))

    return [
        {"file": os.path.basename(fp), "path": fp, "prob": float(prob_avg[i])}
        for i, fp in enumerate(files)
    ]

@torch.no_grad()
def multi_experiment_predict_probs_graph(
    model_ctor,
    gat_bool: bool,
    ckpt_groups: list[list[str]],   # len=5, each is list of ckpts for that experiment
    files: list[str],
    batch_size: int = 16,
    num_workers: int = 0,
    return_std: bool = True,
) -> list[dict]:
    """
    For each experiment:
      - average probs across that experiment's submodels (your current ensemble)
    Then:
      - average across experiments (and optionally std across experiments)

    Returns rows: {file, path, prob, prob_std(optional)}
    """
    if len(files) == 0:
        return []

    # We'll store experiment-level predicted probs [n_exps, n_files]
    exp_probs = []

    for exp_i, ckpt_paths in enumerate(ckpt_groups, start=1):
        print(f"\n[EXP {exp_i}/{len(ckpt_groups)}] {len(ckpt_paths)} submodels")
        rows = ensemble_predict_probs_graph(
            model_ctor=model_ctor,
            gat_bool=gat_bool,
            ckpt_paths=ckpt_paths,
            files=files,
            batch_size=batch_size,
            num_workers=num_workers,
        )
        exp_probs.append(np.array([r["prob"] for r in rows], dtype=np.float64))

    exp_probs = np.stack(exp_probs, axis=0)  # [n_exps, n_files]
    mean_probs = exp_probs.mean(axis=0)

    if return_std:
        std_probs = exp_probs.std(axis=0, ddof=0)
    else:
        std_probs = None

    out = []
    for i, fp in enumerate(files):
        row = {"file": os.path.basename(fp), "path": fp, "prob": float(mean_probs[i])}
        if std_probs is not None:
            row["prob_std"] = float(std_probs[i])
        out.append(row)

    return out

@torch.no_grad()
def multi_experiment_predict_probs_cnn2d(
    ckpt_groups: list[list[str]],   # len=5
    files: list[str],
    batch_size: int = 32,
    cnn_h: int = 200,
    cnn_w: int = 65,
    return_std: bool = True,
) -> list[dict]:
    """
    Same idea as graph wrapper:
      - avg across submodels within each experiment
      - then avg across experiments (optionally std)
    """
    if len(files) == 0:
        return []

    exp_probs = []
    for exp_i, ckpt_paths in enumerate(ckpt_groups, start=1):
        print(f"\n[EXP {exp_i}/{len(ckpt_groups)}] {len(ckpt_paths)} submodels")
        rows = ensemble_predict_probs_cnn2d(
            ckpt_paths=ckpt_paths,
            files=files,
            batch_size=batch_size,
            cnn_h=cnn_h,
            cnn_w=cnn_w,
        )
        exp_probs.append(np.array([r["prob"] for r in rows], dtype=np.float64))

    exp_probs = np.stack(exp_probs, axis=0)  # [n_exps, n_files]
    mean_probs = exp_probs.mean(axis=0)

    if return_std:
        std_probs = exp_probs.std(axis=0, ddof=0)
    else:
        std_probs = None

    out = []
    for i, fp in enumerate(files):
        row = {"file": os.path.basename(fp), "path": fp, "prob": float(mean_probs[i])}
        if std_probs is not None:
            row["prob_std"] = float(std_probs[i])
        out.append(row)

    return out

# -----------------------------
# Ensemble inference (CNN2D)
# -----------------------------
def encoded_matrix_to_cnn_input(encoded_matrix: np.ndarray, cnn_h: int, cnn_w: int) -> torch.Tensor:
    """
    MUST match how you trained CNN2D.

    Here we assume 1-channel images of shape (cnn_h, cnn_w).
    We reshape only if total size matches.
    """
    if encoded_matrix.ndim != 2:
        raise ValueError(f"encoded_matrix must be 2D, got shape {encoded_matrix.shape}")

    img = encoded_matrix
    if img.shape != (cnn_h, cnn_w):
        if img.size != cnn_h * cnn_w:
            raise ValueError(
                f"encoded_matrix shape {img.shape} (size={img.size}) cannot reshape to ({cnn_h},{cnn_w}). "
                f"Update cnn_h/cnn_w to your training shape."
            )
        img = img.reshape(cnn_h, cnn_w)

    return torch.tensor(img, dtype=torch.float32).unsqueeze(0)  # [C=1,H,W]


@torch.no_grad()
def ensemble_predict_probs_cnn2d(
    ckpt_paths: list[str],
    files: list[str],
    batch_size: int = 32,
    cnn_h: int = 200,
    cnn_w: int = 65,
) -> list[dict]:
    """
    Averages PROBABILITIES across CNN2D submodels.
    """
    if len(files) == 0:
        return []

    prob_sum = np.zeros(len(files), dtype=np.float64)

    for k, ckpt in enumerate(ckpt_paths):
        model = CNN2D(input_channels=1).to(device)
        model = load_state_dict_into(model, ckpt)
        model.eval()

        for i in range(0, len(files), batch_size):
            batch_files = files[i:i+batch_size]
            xs = []
            for fp in batch_files:
                enc = load_cnn_matrix(fp)
                xs.append(encoded_matrix_to_cnn_input(enc, cnn_h=cnn_h, cnn_w=cnn_w))
            x_batch = torch.stack(xs, dim=0).to(device)  # [B,1,H,W]

            logits = model(x_batch).view(-1)
            probs = torch.sigmoid(logits).detach().cpu().numpy()
            prob_sum[i:i+len(probs)] += probs

        print(f"  submodel {k+1}/{len(ckpt_paths)} done: {os.path.basename(ckpt)}")

    prob_avg = prob_sum / float(len(ckpt_paths))
    rows = []
    for i, fp in enumerate(files):
        rows.append({
            "file": os.path.basename(fp),
            "path": fp,
            "prob": float(prob_avg[i]),
        })
    return rows

# -----------------------------
# Run PU eval for a given ensemble
# -----------------------------
def run_pu_eval_ensemble(
    model_name: str,
    out_dir: str,
    spies_dir: str,
    eval_dir: str,
    predict_rows_fn,     # function(files)->rows
):
    ensure_dir(out_dir)

    spy_files = list_npy_files(spies_dir)
    eval_files = list_npy_files(eval_dir)

    if len(spy_files) == 0:
        raise FileNotFoundError(f"[{model_name}] no spies in {spies_dir}")
    if len(eval_files) == 0:
        raise FileNotFoundError(f"[{model_name}] no eval files in {eval_dir}")

    print(f"\n[{model_name}] Predicting spies ({len(spy_files)}) ...")
    spy_rows = predict_rows_fn(spy_files)
    spy_probs = np.array([r["prob"] for r in spy_rows], dtype=float)

    spy_stats = compute_spy_percentiles(spy_probs)
    spy_stats["model"] = model_name
    spy_stats["spies_dir"] = spies_dir

    # save spies stats + spies probs
    with open(os.path.join(out_dir, f"{model_name}_spies_stats.json"), "w") as f:
        json.dump(spy_stats, f, indent=2)
    pd.DataFrame([spy_stats]).to_csv(os.path.join(out_dir, f"{model_name}_spies_stats.csv"), index=False)
    pd.DataFrame(spy_rows).to_csv(os.path.join(out_dir, f"{model_name}_spies_probs.csv"), index=False)

    print(f"[{model_name}] spies thresholds: p25={spy_stats['p25']:.4f}, p50={spy_stats['p50']:.4f}, p75={spy_stats['p75']:.4f}")

    print(f"\n[{model_name}] Predicting eval dir ({len(eval_files)}) ...")
    eval_rows = predict_rows_fn(eval_files)

    # label using spies thresholds
    for r in eval_rows:
        r["label"] = label_from_percentiles(r["prob"], spy_stats)

    df_eval = pd.DataFrame(eval_rows)
    out_csv = os.path.join(out_dir, f"{model_name}_eval_probs_and_labels.csv")
    df_eval.to_csv(out_csv, index=False)

    out_png = os.path.join(out_dir, f"{model_name}_eval_label_hist.png")
    plot_label_histogram(df_eval, f"{model_name} label counts (ensemble avg; thresholds from spies)", out_png)

    print(f"\n[{model_name}] label counts:\n{df_eval['label'].value_counts()}")
    print(f"Saved: {out_csv}")
    print(f"Saved: {out_png}")
    print(f"Saved: {os.path.join(out_dir, f'{model_name}_spies_stats.json')}\n")

In [8]:
# Each entry = one experiment's checkpoints (submodels)
GAT_EXPS = [
    "../CLR_Ligand_Training/GAT-CLR_Exp1/Models/*.pth",
    "../CLR_Ligand_Training/GAT-CLR_Exp2/Models/*.pth",
    "../CLR_Ligand_Training/GAT-CLR_Exp3/Models/*.pth",
    "../CLR_Ligand_Training/GAT-CLR_Exp4/Models/*.pth",
    "../CLR_Ligand_Training/GAT-CLR_Exp5/Models/*.pth",
]

GCN_EXPS = [
    "../CLR_Ligand_Training/GCN-CLR_Exp1/*.pth",
    "../CLR_Ligand_Training/GCN-CLR_Exp2/*.pth",
    "../CLR_Ligand_Training/GCN-CLR_Exp3/*.pth",
    "../CLR_Ligand_Training/GCN-CLR_Exp4/*.pth",
    "../CLR_Ligand_Training/GCN-CLR_Exp5/*.pth",
]

CNN2D_EXPS = [
    "../CLR_Ligand_Training/GNN-CLR_Exp1/*.pth",
    "../CLR_Ligand_Training/GNN-CLR_Exp2/*.pth",
    "../CLR_Ligand_Training/GNN-CLR_Exp3/*.pth",
    "../CLR_Ligand_Training/GNN-CLR_Exp4/*.pth",
    "../CLR_Ligand_Training/GNN-CLR_Exp5/*.pth",
]

CNN3D_EXPS = [
    "../CLR_Ligand_Training/3DCNN-CLR_Exp1/*.pth",
    "../CLR_Ligand_Training/3DCNN-CLR_Exp2/*.pth"
]

gat_ckpt_groups = [resolve_ensemble_checkpoints(p) for p in GAT_EXPS]
gcn_ckpt_groups = [resolve_ensemble_checkpoints(p) for p in GCN_EXPS]
cnn_ckpt_groups = [resolve_ensemble_checkpoints(p) for p in CNN2D_EXPS]
cnn3d_ckpt_groups = [resolve_ensemble_checkpoints(p) for p in CNN3D_EXPS]

print("GAT experiments:", [len(g) for g in gat_ckpt_groups])
print("GCN experiments:", [len(g) for g in gcn_ckpt_groups])
print("CNN2D experiments:", [len(g) for g in cnn_ckpt_groups])
print("CNN3D experiments:", [len(g) for g in cnn3d_ckpt_groups])

GRAPH_INPUT_DIM = infer_graph_input_dim(gat_gcn_spies_exps[0])

def make_gat():
    return GAT(in_channels=GRAPH_INPUT_DIM, out_channels=32, dropout_p=0.1)

def make_gcn():
    return GCN(input_dim=GRAPH_INPUT_DIM)

# run_pu_eval_5exp_with_exp_spies(
#     model_name="GAT_ENSEMBLE",
#     out_dir=os.path.join(OUT_ROOT, "GAT_5_exp_separate_graphs_sep"),
#     spies_dirs=gat_gcn_spies_exps,
#     eval_dir=gat_gcn_unlabeled,
#     predict_rows_fn_per_exp=lambda files, exp_idx: ensemble_predict_probs_graph(
#         model_ctor=make_gat,
#         gat_bool=True,
#         ckpt_paths=gat_ckpt_groups[exp_idx],   # <-- exp-specific ckpts
#         files=files,
#         batch_size=16,
#     ),
# )

# run_pu_eval_5exp_with_exp_spies(
#     model_name="GCN_ENSEMBLE",
#     out_dir=os.path.join(OUT_ROOT, "GCN_5_exp_separate_graphs_sep"),
#     spies_dirs=gat_gcn_spies_exps,
#     eval_dir=gat_gcn_unlabeled,
#     predict_rows_fn_per_exp=lambda files, exp_idx: ensemble_predict_probs_graph(
#         model_ctor=make_gcn,
#         gat_bool=False,
#         ckpt_paths=gcn_ckpt_groups[exp_idx],   # <-- exp-specific ckpts
#         files=files,
#         batch_size=16,
#     ),
# )

# run_pu_eval_5exp_with_exp_spies(
#     model_name="CNN2D_ENSEMBLE",
#     out_dir=os.path.join(OUT_ROOT, "CNN2D_5_exp_graph_format_sep"),
#     spies_dirs=gnn_spies_exps,
#     eval_dir=gnn_unlabeled,
#     predict_rows_fn_per_exp=lambda files, exp_idx: ensemble_predict_probs_cnn2d(
#         ckpt_paths=cnn_ckpt_groups[exp_idx],  # <-- exp-specific ckpts
#         files=files,
#         batch_size=32,
#         cnn_h=200,
#         cnn_w=65,
#     ),
# )

run_pu_eval_5exp_with_exp_spies(
    model_name="CNN3D_ENSEMBLE",
    out_dir=os.path.join(OUT_ROOT, "CNN3D_2_exp_total_grid_sep"),
    spies_dirs=cnn_spies_exps,          # <-- 3D CNN spies should match grid-format experiments
    eval_dir=cnn_unlabeled,             # <-- IMPORTANT: this must point to your 3D GRID unlabeled dir
    predict_rows_fn_per_exp=lambda files, exp_idx: ensemble_predict_probs_cnn3d(
        ckpt_paths=cnn3d_ckpt_groups[exp_idx],
        files=files,
        batch_size=8,
    ),
)


GAT experiments: [50, 50, 50, 50, 50]
GCN experiments: [50, 50, 50, 50, 50]
CNN2D experiments: [50, 50, 50, 50, 50]
CNN3D experiments: [50, 50]

[CNN3D_ENSEMBLE] EXP 1/2: spies=770 eval=75
  submodel 1/50 done: model_bin_1.pth
  submodel 2/50 done: model_bin_10.pth
  submodel 3/50 done: model_bin_11.pth
  submodel 4/50 done: model_bin_12.pth
  submodel 5/50 done: model_bin_13.pth
  submodel 6/50 done: model_bin_14.pth
  submodel 7/50 done: model_bin_15.pth
  submodel 8/50 done: model_bin_16.pth
  submodel 9/50 done: model_bin_17.pth
  submodel 10/50 done: model_bin_18.pth
  submodel 11/50 done: model_bin_19.pth
  submodel 12/50 done: model_bin_2.pth
  submodel 13/50 done: model_bin_20.pth
  submodel 14/50 done: model_bin_21.pth
  submodel 15/50 done: model_bin_22.pth
  submodel 16/50 done: model_bin_23.pth
  submodel 17/50 done: model_bin_24.pth
  submodel 18/50 done: model_bin_25.pth
  submodel 19/50 done: model_bin_26.pth
  submodel 20/50 done: model_bin_27.pth
  submodel 21/50 done:

In [ ]:
import re
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# =========================
# Config
# =========================
BASE = Path(".")

# 1_16 results
MODEL_FILES_1_16 = {
    "GNN": BASE / "PU_EvalOutputs_Ensemble/1_16_results/CNN2D_5_exp_graph_format_1_16/CNN2D_ENSEMBLE_eval_probs_and_labels.csv",
    "GAT": BASE / "PU_EvalOutputs_Ensemble/1_16_results/GAT_5_exp_separate_graphs_1_16/GAT_ENSEMBLE_eval_probs_and_labels.csv",
    "GCN": BASE / "PU_EvalOutputs_Ensemble/1_16_results/GCN_5_exp_separate_graphs_1_16/GCN_ENSEMBLE_eval_probs_and_labels.csv",
}
OCC_PREFIX_1_16 = "1_16-piezo-graph-5A"

# sep results
MODEL_FILES_SEP = {
    "GNN": BASE / "PU_EvalOutputs_Ensemble/sep_results/CNN2D_5_exp_graph_format_sep/CNN2D_ENSEMBLE_eval_probs_and_labels.csv",
    "GAT": BASE / "PU_EvalOutputs_Ensemble/sep_results/GAT_5_exp_separate_graphs_sep/GAT_ENSEMBLE_eval_probs_and_labels.csv",
    "GCN": BASE / "PU_EvalOutputs_Ensemble/sep_results/GCN_5_exp_separate_graphs_sep/GCN_ENSEMBLE_eval_probs_and_labels.csv",
}
OCC_PREFIX_SEP = "separate-piezo-graph-5A"

# anton results (NEW)
MODEL_FILES_ANTON = {
    "GNN": BASE / "PU_EvalOutputs_Ensemble/anton_results/CNN2D_5_exp_graph_format_anton/CNN2D_ENSEMBLE_eval_probs_and_labels.csv",
    "GAT": BASE / "PU_EvalOutputs_Ensemble/anton_results/GAT_5_exp_separate_graphs_anton/GAT_ENSEMBLE_eval_probs_and_labels.csv",
    "GCN": BASE / "PU_EvalOutputs_Ensemble/anton_results/GCN_5_exp_separate_graphs_anton/GCN_ENSEMBLE_eval_probs_and_labels.csv",
}
OCC_PREFIX_ANTON = "anton-piezo-graph-5A"

OCC_FILE = BASE / "Alex_MD_data.csv"

# =========================
# Dynamic binning config
# =========================
XTICKS = np.array([0, 20, 40, 60, 80, 100])

BINNING_MODE = "quantile"   # "quantile" or "equal_width"
N_BINS = 8                  # increase this for more bins (e.g., 6, 8, 10, 12)

# Limit binning to this occupancy range (useful if you want to ignore extreme tails)
BIN_RANGE = (0.0, 100.0)    # (min, max)


# =========================
# Helpers
# =========================
def make_global_id(dataset_label: str, local_id: str) -> str:
    return f"{dataset_label}::{local_id}"

def extract_id_from_eval_row(file_val: str, path_val: str) -> str | None:
    s_file = str(file_val)
    s_path = str(path_val)

    # 1_16 / sep: "2828_graphs.npy" -> "2828"
    m = re.match(r"(\d{4})", s_file)
    if m:
        return m.group(1)

    # anton: want "box13_mode8" etc (try file first)
    m = re.search(r"(box\d+).*?(?:mode[_-]?(\d+))", s_file, flags=re.IGNORECASE)
    if m:
        return f"{m.group(1).lower()}_mode{m.group(2)}"

    # if file doesn't contain mode, try path
    m = re.search(r"(box\d+).*?(?:mode[_-]?(\d+))", s_path, flags=re.IGNORECASE)
    if m:
        return f"{m.group(1).lower()}_mode{m.group(2)}"

    # fallback: sometimes only "box#" exists (but this will create collisions)
    m = re.search(r"(box\d+)", s_file, flags=re.IGNORECASE)
    if m:
        return m.group(1).lower()

    m = re.search(r"(box\d+)", s_path, flags=re.IGNORECASE)
    if m:
        return m.group(1).lower()

    return None

def extract_id_from_occ_filename(dataset_label: str, filename_val: str) -> str | None:
    s = str(filename_val)

    if dataset_label in ("1_16", "sep"):
        m = re.search(r"CHL1_(\d{4})", s)
        return m.group(1) if m else None

    if dataset_label == "anton":
        m = re.search(r"(box\d+).*?(?:mode[_-]?(\d+))", s, flags=re.IGNORECASE)
        if m:
            return f"{m.group(1).lower()}_mode{m.group(2)}"
        return None

    return None

def load_model_eval_csv(path: Path, dataset_label: str) -> pd.DataFrame:
    df = pd.read_csv(path)

    local_ids = [
        extract_id_from_eval_row(f, p) for f, p in zip(df["file"], df["path"])
    ]
    df["id_key"] = [
        make_global_id(dataset_label, lid) if lid is not None else None
        for lid in local_ids
    ]

    df = df.dropna(subset=["id_key"]).copy()
    df["prob"] = pd.to_numeric(df["prob"], errors="coerce")
    df = df.dropna(subset=["prob"]).copy()
    df["dataset"] = dataset_label
    return df

def load_occupancy_csv(path: Path, prefixes: list[str]) -> pd.DataFrame:
    occ = pd.read_csv(path)

    fn = occ["filename"].astype(str)
    mask = False
    for p in prefixes:
        mask = mask | fn.str.startswith(p)
    occ = occ[mask].copy()

    def dataset_from_prefix(x: str) -> str:
        s = str(x)
        if s.startswith(OCC_PREFIX_1_16):
            return "1_16"
        if s.startswith(OCC_PREFIX_SEP):
            return "sep"
        if s.startswith(OCC_PREFIX_ANTON):
            return "anton"
        return "unknown"

    occ["occ_dataset"] = occ["filename"].astype(str).apply(dataset_from_prefix)

    local_ids = [
        extract_id_from_occ_filename(dataset, fname)
        for dataset, fname in zip(occ["occ_dataset"], occ["filename"])
    ]
    occ["id_key"] = [
        make_global_id(dataset, lid) if lid is not None else None
        for dataset, lid in zip(occ["occ_dataset"], local_ids)
    ]
    occ = occ.dropna(subset=["id_key"]).copy()

    # numeric columns
    for col in ["high_occupancy", "GNN", "GAT", "GCN"]:
        if col in occ.columns:
            occ[col] = pd.to_numeric(occ[col], errors="coerce")

    occ = occ.dropna(subset=["high_occupancy"]).copy()

    # keep first per id_key
    occ = occ.sort_values(["id_key", "occ_dataset"]).drop_duplicates("id_key", keep="first")

    # keep old scores too (if present)
    keep_cols = ["id_key", "high_occupancy", "occ_dataset", "filename"]
    for c in ["GNN", "GAT", "GCN", "gnn_std", "gat_std", "gcn_std"]:
        if c in occ.columns:
            keep_cols.append(c)

    return occ[keep_cols]


def compute_dynamic_bins(occ_values: pd.Series, mode: str, n_bins: int, bin_range=(0.0, 100.0)):
    """
    Returns (bin_edges, bin_labels, bin_pos, bin_widths)
    Edges live in the same units as high_occupancy.
    """
    v = pd.to_numeric(occ_values, errors="coerce").dropna().astype(float)
    v = v.clip(lower=bin_range[0], upper=bin_range[1])

    if v.empty:
        raise ValueError("No occupancy values available to compute bins.")

    if mode == "quantile":
        qs = np.linspace(0, 1, n_bins + 1)
        edges = np.quantile(v, qs)
        edges = np.unique(edges)  # remove duplicates if many identical values

        # If duplicates collapse bins, fall back to equal-width
        if len(edges) < 4:
            edges = np.linspace(bin_range[0], bin_range[1], n_bins + 1)

    elif mode == "equal_width":
        edges = np.linspace(bin_range[0], bin_range[1], n_bins + 1)

    else:
        raise ValueError(f"Unknown BINNING_MODE: {mode}")

    # Ensure strictly increasing edges
    edges = np.array(edges, dtype=float)
    edges = np.unique(edges)
    if len(edges) < 3:
        raise ValueError("Computed bin edges collapsed too much. Reduce N_BINS or use equal_width.")

    labels = [f"{edges[i]:.1f}–{edges[i+1]:.1f}" for i in range(len(edges) - 1)]
    pos = (edges[:-1] + edges[1:]) / 2.0
    widths = (edges[1:] - edges[:-1])

    return edges, labels, pos, widths



def assign_bins(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["high_occupancy"] = df["high_occupancy"].clip(lower=0, upper=100)

    df["occ_bin"] = pd.cut(
        df["high_occupancy"],
        bins=BIN_EDGES,
        include_lowest=True,
        right=True,
        labels=BIN_LABELS
    )

    df = df.dropna(subset=["occ_bin"]).copy()
    df["occ_bin"] = df["occ_bin"].astype(
        pd.CategoricalDtype(categories=BIN_LABELS, ordered=True)
    )
    return df



def build_model_dfs(model_files: dict, dataset_label: str, occ: pd.DataFrame) -> dict[str, pd.DataFrame]:
    out = {}
    for model_name, csv_path in model_files.items():
        mdf = load_model_eval_csv(csv_path, dataset_label=dataset_label)
        mdf = mdf.merge(occ, on="id_key", how="inner")
        mdf = assign_bins(mdf)
        out[model_name] = mdf
    return out


# =========================
# Load occupancy (all 3 prefixes)
# =========================
occ = load_occupancy_csv(
    OCC_FILE,
    prefixes=[OCC_PREFIX_1_16, OCC_PREFIX_SEP, OCC_PREFIX_ANTON]
)

# =========================
# Compute dynamic bins from OCC values (one time)
# =========================
BIN_EDGES, BIN_LABELS, BIN_POS, BIN_WIDTHS = compute_dynamic_bins(
    occ_values=occ["high_occupancy"],
    mode=BINNING_MODE,
    n_bins=N_BINS,
    bin_range=BIN_RANGE
)

print("\nDynamic bins:")
for i in range(len(BIN_LABELS)):
    print(f"  Bin {i+1}: {BIN_LABELS[i]}")


# =========================
# Load each dataset + merge with occupancy
# =========================
dfs_1_16 = build_model_dfs(MODEL_FILES_1_16, dataset_label="1_16", occ=occ)
dfs_sep  = build_model_dfs(MODEL_FILES_SEP,  dataset_label="sep",  occ=occ)
dfs_anton= build_model_dfs(MODEL_FILES_ANTON,dataset_label="anton",occ=occ)

# =========================
# Combine 1_16 + sep + anton into ONE dataset per model
# =========================
models_in_order = ["GNN", "GAT", "GCN"]

dfs_all = {}
for model_name in models_in_order:
    dfs_all[model_name] = pd.concat(
        [dfs_1_16[model_name], dfs_sep[model_name], dfs_anton[model_name]],
        ignore_index=True
    )

# =========================
# Build OLD (from OCC columns) and align to NEW ids
# =========================
MODEL_TO_OCC_COL = {"GNN": "GNN", "GAT": "GAT", "GCN": "GCN"}

dfs_old = {}
for model_name in models_in_order:
    col = MODEL_TO_OCC_COL[model_name]
    if col not in occ.columns:
        print(f"[WARN] OCC column '{col}' not found. Skipping OLD for {model_name}.")
        continue

    # keep rows that have an old score for this model
    old_df = occ.dropna(subset=[col]).copy()
    old_df = old_df.rename(columns={col: "prob_old"})
    old_df["prob_old"] = pd.to_numeric(old_df["prob_old"], errors="coerce")
    old_df = old_df.dropna(subset=["prob_old"]).copy()

    # align OLD to the ids that exist in NEW for this model (apples-to-apples)
    new_ids = set(dfs_all[model_name]["id_key"])
    old_df = old_df[old_df["id_key"].isin(new_ids)].copy()

    # add bins using same BIN_EDGES/BIN_LABELS
    old_df["occ_bin"] = pd.cut(
        old_df["high_occupancy"].clip(0, 100),
        bins=BIN_EDGES,
        include_lowest=True,
        right=True,
        labels=BIN_LABELS
    )
    old_df = old_df.dropna(subset=["occ_bin"]).copy()
    old_df["occ_bin"] = old_df["occ_bin"].astype(
        pd.CategoricalDtype(categories=BIN_LABELS, ordered=True)
    )

    dfs_old[model_name] = old_df

# =========================
# Rank-Pearson correlation: Pearson(ranks)
# (This is Spearman's rho)
# Filter: occupancy > 20%
# =========================

OCC_THRESHOLD = 30.0  # <-- THIS is what your boss asked for

def pearson_r(x: np.ndarray, y: np.ndarray) -> float:
    """Standard Pearson correlation on raw values."""
    x = pd.Series(x).astype(float)
    y = pd.Series(y).astype(float)
    return float(x.corr(y, method="pearson"))

def rank_pearson_r(x: np.ndarray, y: np.ndarray, tie_method: str = "average") -> float:
    """
    "Ranking Pearson correlation" = Pearson correlation of the ranks.
    This is Spearman's rho by definition.
    tie_method: 'average' (default), 'min', 'max', 'dense', 'first'
    """
    x = pd.Series(x).astype(float).rank(method=tie_method)
    y = pd.Series(y).astype(float).rank(method=tie_method)
    return float(x.corr(y, method="pearson"))

print("\n==============================")
print(f"Rank-Pearson (Pearson of ranks) and Pearson (raw), filtering occupancy > {OCC_THRESHOLD}%")
print("==============================")

rows = []
aligned_per_model = {}

for model_name in models_in_order:
    # NEW
    new_df = dfs_all[model_name][["id_key", "high_occupancy", "prob"]].copy()
    new_df = new_df.rename(columns={"prob": "prob_new"})
    new_df["high_occupancy"] = pd.to_numeric(new_df["high_occupancy"], errors="coerce")
    new_df["prob_new"] = pd.to_numeric(new_df["prob_new"], errors="coerce")
    new_df = new_df.dropna(subset=["high_occupancy", "prob_new"]).copy()

    # OLD
    old_df = dfs_old[model_name][["id_key", "high_occupancy", "prob_old"]].copy()
    old_df["high_occupancy"] = pd.to_numeric(old_df["high_occupancy"], errors="coerce")
    old_df["prob_old"] = pd.to_numeric(old_df["prob_old"], errors="coerce")
    old_df = old_df.dropna(subset=["high_occupancy", "prob_old"]).copy()

    # Align by id_key
    merged = pd.merge(old_df, new_df[["id_key", "prob_new"]], on="id_key", how="inner")

    # Apply occupancy filter
    merged = merged[merged["high_occupancy"] > OCC_THRESHOLD].copy()

    aligned_per_model[model_name] = merged

    if len(merged) < 3:
        pear_old = pear_new = np.nan
        rpear_old = rpear_new = np.nan
        rpear_old_vs_new = np.nan
        n = len(merged)
    else:
        x = merged["high_occupancy"].values
        y_old = merged["prob_old"].values
        y_new = merged["prob_new"].values

        # raw Pearson (optional)
        pear_old = pearson_r(x, y_old)
        pear_new = pearson_r(x, y_new)

        # "ranking Pearson" (what was requested)
        rpear_old = rank_pearson_r(x, y_old)   # = Spearman(occ, old_score)
        rpear_new = rank_pearson_r(x, y_new)   # = Spearman(occ, new_score)

        # agreement between old vs new scores in rank space (often useful)
        rpear_old_vs_new = rank_pearson_r(y_old, y_new)
        n = len(merged)

    rows.append({
        "model": model_name,
        "n_filtered_aligned": int(n),

        # Raw Pearson
        "pearson_old_raw": pear_old,
        "pearson_new_raw": pear_new,

        # Rank-Pearson (Pearson of ranks) == Spearman
        "rankPearson_old": rpear_old,
        "rankPearson_new": rpear_new,
        "rankPearson_old_vs_new": rpear_old_vs_new,
    })

    print(f"\n{model_name}: n={n}")
    if n < 3:
        print("  (n<3) correlations not computed")
    else:
        print(f"  RAW Pearson(occ, OLD score): {pear_old:.4f}")
        print(f"  RAW Pearson(occ, NEW score): {pear_new:.4f}")
        print(f"  Rank-Pearson(occ, OLD score): {rpear_old:.4f}  (Pearson of ranks)")
        print(f"  Rank-Pearson(occ, NEW score): {rpear_new:.4f}  (Pearson of ranks)")
        print(f"  Rank-Pearson(OLD score, NEW score): {rpear_old_vs_new:.4f}")

summary_df = pd.DataFrame(rows)
print("\nSummary:")
print(summary_df.to_string(index=False))


# =========================
# Rank vs Rank scatter plots (occupancy vs score)
# OLD on top row, NEW on bottom row
# =========================

fig, axes = plt.subplots(
    nrows=2,
    ncols=len(models_in_order),
    figsize=(22, 10),
    sharex=False,
    sharey="row"
)

if len(models_in_order) == 1:
    axes = np.array([[axes[0]], [axes[1]]])

for col_idx, model_name in enumerate(models_in_order):
    merged = aligned_per_model.get(model_name, pd.DataFrame())

    ax_old = axes[0, col_idx]
    ax_new = axes[1, col_idx]

    ax_old.set_title(f"{model_name} — OLD (rank vs rank)")
    ax_new.set_title(f"{model_name} — NEW (rank vs rank)")

    if merged is None or merged.empty or len(merged) < 3:
        ax_old.text(0.5, 0.5, "Insufficient data", ha="center", va="center", transform=ax_old.transAxes)
        ax_new.text(0.5, 0.5, "Insufficient data", ha="center", va="center", transform=ax_new.transAxes)
        continue

    # Raw values
    x = merged["high_occupancy"].values
    y_old = merged["prob_old"].values
    y_new = merged["prob_new"].values

    # Ranks
    x_rank = pd.Series(x).rank(method="average").values
    y_old_rank = pd.Series(y_old).rank(method="average").values
    y_new_rank = pd.Series(y_new).rank(method="average").values

    # Scatter
    ax_old.scatter(x_rank, y_old_rank, s=18, alpha=0.7)
    ax_new.scatter(x_rank, y_new_rank, s=18, alpha=0.7)

    # Rank-Pearson correlations
    r_old = rank_pearson_r(x, y_old)
    r_new = rank_pearson_r(x, y_new)

    ax_old.text(
        0.02, 0.95,
        f"rank-Pearson r = {r_old:.3f}\nn = {len(merged)}",
        transform=ax_old.transAxes,
        ha="left", va="top"
    )

    ax_new.text(
        0.02, 0.95,
        f"rank-Pearson r = {r_new:.3f}\nn = {len(merged)}",
        transform=ax_new.transAxes,
        ha="left", va="top"
    )

    # Axis labels
    if col_idx == 0:
        ax_old.set_ylabel("Rank(score) — OLD")
        ax_new.set_ylabel("Rank(score) — NEW")

    ax_new.set_xlabel("Rank(residue occupancy)")

    for ax in [ax_old, ax_new]:
        ax.grid(True, alpha=0.25)

plt.tight_layout()
plt.show()

# =========================
# Find OCC samples missing from model results
# =========================

# All OCC ids
occ_ids = set(occ["id_key"])

# All ids present in any model (union across models)
model_ids = set()
for model_name in models_in_order:
    model_ids |= set(dfs_all[model_name]["id_key"])

missing_ids = sorted(occ_ids - model_ids)

print("\n==============================")
print("OCC samples NOT found in any model results")
print("==============================")
print(f"Missing count: {len(missing_ids)}")

for mid in missing_ids:
    print(mid)

# =========================
# Plot: OLD (top) vs NEW (bottom) per model
# =========================
fig, axes = plt.subplots(
    nrows=2,
    ncols=len(models_in_order),
    figsize=(22, 10),
    sharex=True,
    sharey="row"
)

if len(models_in_order) == 1:
    axes = np.array([[axes[0]], [axes[1]]])

colors = ["#FF6B6B", "#FFA36B", "#6BFF8D", "#6BA3FF"]  # GNN, GAT, 3D CNN, GCN
MODEL_COLORS = {
    "GNN": colors[0],
    "GAT": colors[1],
    "GCN": colors[3],
}

for col_idx, model_name in enumerate(models_in_order):

    color = MODEL_COLORS[model_name]

    # =====================
    # TOP: OLD (from OCC)
    # =====================
    ax_old = axes[0, col_idx]
    old_df = dfs_old.get(model_name)

    old_data = [
        old_df.loc[old_df["occ_bin"] == b, "prob_old"].values
        for b in BIN_LABELS
    ]
    old_counts = [len(a) for a in old_data]
    old_means = [np.mean(a) if len(a) else np.nan for a in old_data]

    # bars behind
    ax2_old = ax_old.twinx()
    ax2_old.bar(
        BIN_POS,
        old_counts,
        width=BIN_WIDTHS,
        alpha=0.25,
        align="center",
        zorder=0
    )
    ax2_old.set_ylim(0, max([1] + old_counts) * 1.15)
    ax2_old.set_ylabel("# samples (OLD)")

    bp_old = ax_old.boxplot(
        old_data,
        positions=BIN_POS,
        widths=BIN_WIDTHS,
        patch_artist=True,
        showfliers=True,
        manage_ticks=False,
        zorder=3
    )

    for box in bp_old["boxes"]:
        box.set_facecolor(color)
        box.set_edgecolor("black")
        box.set_alpha(0.6)

    for k in ["whiskers", "caps", "medians"]:
        for line in bp_old[k]:
            line.set_color("black")
    for m in bp_old["medians"]:
        m.set_linewidth(2)

    ax_old.plot(
        BIN_POS, old_means,
        marker="o", linestyle="--", linewidth=2,
        label="OLD mean", zorder=4
    )

    ax_old.set_title(f"{model_name} — OLD")
    if col_idx == 0:
        ax_old.set_ylabel("Probability score")

    # =====================
    # BOTTOM: NEW (Ensemble)
    # =====================
    ax_new = axes[1, col_idx]
    new_df = dfs_all[model_name]

    new_data = [
        new_df.loc[new_df["occ_bin"] == b, "prob"].values
        for b in BIN_LABELS
    ]
    new_counts = [len(a) for a in new_data]
    new_means = [np.mean(a) if len(a) else np.nan for a in new_data]

    ax2_new = ax_new.twinx()
    ax2_new.bar(
        BIN_POS,
        new_counts,
        width=BIN_WIDTHS,
        alpha=0.25,
        align="center",
        zorder=0
    )
    ax2_new.set_ylim(0, max([1] + new_counts) * 1.15)
    ax2_new.set_ylabel("# samples (NEW)")

    bp_new = ax_new.boxplot(
        new_data,
        positions=BIN_POS,
        widths=BIN_WIDTHS,
        patch_artist=True,
        showfliers=True,
        manage_ticks=False,
        zorder=3
    )

    for box in bp_new["boxes"]:
        box.set_facecolor(color)
        box.set_edgecolor("black")
        box.set_alpha(0.85)

    for k in ["whiskers", "caps", "medians"]:
        for line in bp_new[k]:
            line.set_color("black")
    for m in bp_new["medians"]:
        m.set_linewidth(2)

    ax_new.plot(
        BIN_POS, new_means,
        marker="o", linestyle="-", linewidth=2,
        label="NEW mean", zorder=4
    )

    ax_new.set_title(f"{model_name} — NEW")
    ax_new.set_xlabel("Residue occupancy")
    ax_new.set_xlim(0, 100)
    ax_new.set_xticks(XTICKS)
    ax_new.set_xticklabels([str(int(t)) for t in XTICKS])

    if col_idx == 0:
        ax_new.set_ylabel("Probability score")

    # vertical bin boundaries
    for edge in BIN_EDGES:
        ax_old.axvline(edge, linestyle="--", linewidth=0.8, alpha=0.4)
        ax_new.axvline(edge, linestyle="--", linewidth=0.8, alpha=0.4)

plt.tight_layout()
plt.show()

# =========================
# Sanity prints
# =========================
for model_name in models_in_order:
    print(
        f"{model_name}: 1_16={len(dfs_1_16[model_name])}, "
        f"sep={len(dfs_sep[model_name])}, "
        f"anton={len(dfs_anton[model_name])}, "
        f"combined={len(dfs_all[model_name])}"
    )
